1. 일단 데이터를 날짜별로 자른다.
2. 모든 피처를 다 만드는 경우
  1) 나이대 10개
  2) 성별 2개
  3) 감염 원인 11개
  4) 방문 요일 7개
     총 30개 배열

In [1]:
import pandas as pd
import scipy.stats as st
from datetime import datetime
from datetime import timedelta
import math
import numpy as np
from PIL import Image
from scipy import stats

### 1. 루트 생성

In [2]:
patient_route = pd.read_csv('covid19/PatientRoute.csv')
patient_info = pd.read_csv('covid19/PatientInfo.csv')
patient_info = patient_info[["patient_id","global_num","sex","age","infection_case"]]

In [3]:
merge_route = pd.merge(patient_info, patient_route)
merge_route = merge_route[merge_route["province"]=="Seoul"]
merge_route = merge_route[pd.notnull(merge_route['sex'])]
merge_route = merge_route[pd.notnull(merge_route['age'])]

### 2. 감염 원인 처리

In [4]:
causes = merge_route['infection_case']
causes = causes.drop_duplicates(keep='last')
causes = causes.dropna()
causes = causes.tolist()

causes_stay = ["contact with patient", "overseas inflow", "etc"]
for i, cause in enumerate(causes):
    if cause in causes_stay:
        continue
    causes[i] = "community infection"
    
causes = list(dict.fromkeys(causes))

### 3. 클래스

In [5]:
class GridMap:
    Re = 6371.00877  ##  지도반경
    grid = 0.16  ##  격자간격 (km)
    slat1 = 30.0  ##  표준위도 1
    slat2 = 60.0  ##  표준위도 2
    olon = 127.0175205  ##  기준점 경도
    olat = 37.523124  ##  기준점 위도
    xo = 18.4 / grid #  기준점 X좌표
    yo = 21.6 / grid  ##  기준점 Y좌표
    row = 256
    column = 256
    seoul_start_lat = 37.698098
    seoul_start_lon = 126.799791
    seoul_end_lat = 37.34815
    seoul_end_lon = 127.23525

    def __init__(self):
        self.PI = math.asin(1.0) * 2.0
        self.DEGRAD = self.PI / 180.0
        self.RADDEG = 180.0 / self.PI

        self.re = self.Re / self.grid
        self.slat1 = self.slat1 * self.DEGRAD
        self.slat2 = self.slat2 * self.DEGRAD
        self.olon = self.olon * self.DEGRAD
        self.olat = self.olat * self.DEGRAD

        self.sn = math.tan(self.PI * 0.25 + self.slat2 * 0.5) / \
                  math.tan(self.PI * 0.25 + self.slat1 * 0.5)
        self.sn = math.log(math.cos(self.slat1) /
                           math.cos(self.slat2)) / math.log(self.sn)
        self.sf = math.tan(self.PI * 0.25 + self.slat1 * 0.5)
        self.sf = math.pow(self.sf, self.sn) * math.cos(self.slat1) / self.sn
        self.ro = math.tan(self.PI * 0.25 + self.olat * 0.5)
        self.ro = self.re * self.sf / math.pow(self.ro, self.sn)

    def toGrid(self, lat, lon):
        ra = math.tan(self.PI * 0.25 + lat * self.DEGRAD * 0.5)
        ra = self.re * self.sf / pow(ra, self.sn)
        theta = lon * self.DEGRAD - self.olon
        if theta > self.PI:
            theta -= 2.0 * self.PI
        if theta < self.PI * -1:
            theta += 2.0 * self.PI
        theta *= self.sn
        x = (ra * math.sin(theta)) + self.xo
        y = (self.ro - ra * math.cos(theta)) + self.yo
        x = int(x + 1.5)
        # y = int(y + 1.5)
        y = (int(y + 1.5) - 255) * -1
        return x, y

    def toCoordinate(self, x, y):
        x = x - 1
        y = y - 1
        xn = x - self.xo
        yn = self.ro - y + self.yo
        ra = math.sqrt(xn * xn + yn * yn)
        if self.sn < 0.0:
            ra = -ra
        alat = math.pow((self.re * self.sf / ra), (1.0 / self.sn))
        alat = 2.0 * math.atan(alat) - self.PI * 0.5
        if math.fabs(xn) <= 0.0:
            theta = 0.0
        else:
            if math.fabs(yn) <= 0.0:
                theta = self.PI * 0.5
                if xn < 0.0:
                    theta = -theta
            else:
                theta = math.atan2(xn, yn)
        alon = theta / self.sn + self.olon
        lat = alat * self.RADDEG
        lon = alon * self.RADDEG
        return lat, lon

    def isSeoul(self, lat, lon):
        if self.seoul_end_lat > lat or lat > self.seoul_start_lat:
            return False
        if self.seoul_end_lon < lon or lon < self.seoul_start_lon:
            return False
        return True

    def generateMap(self, raw_data):
        gridx, gridy = [], []
        for idx, data in raw_data.iterrows():
            try:
                float(data.latitude)
            except ValueError:
                continue
            if self.isSeoul(data.latitude, data.longitude):
                x, y = self.toGrid(data.latitude, data.longitude)
                gridx.append(x), gridy.append(y)

        df = pd.DataFrame({'gridx': gridx, 'gridy': gridy})

        grid_array = np.zeros((self.row, self.column))
        for idx, data in df.iterrows():
            try:
                grid_array[data.gridy, data.gridx] += 1
            except:
                pass

        return grid_array

In [6]:
class Feature:
    def __init__(self, names, counts, move_types):
        self.names = names
        self.counts = counts
        self.move_types = move_types
        self.clear(95)
        
    def clear(self, route_dates):
        self.features = []
        all_count = sum(count for count in self.counts)
        self.tensor = np.zeros((route_dates, 256, 256, all_count), dtype=np.float)
        for i in range(all_count):
            arr = np.zeros((256, 256))
            self.features.append(arr)
            
    def age_category(self, ages):
        categories = []
        for age in ages:
            age = int(age[:-1])
            if age == 0: category = 0
            elif age == 100: category = 10
            else: category = age // 10
            categories.append(category)
        return categories
            
    def sex_category(self, sexes):
        categories = []
        for sex in sexes:
            if sex == 'male': categories.append(0)
            else: categories.append(1)
        return categories
        
    def infection_case_category(self, infection_cases):
        categories = []
        for infection_case in infection_cases:
            if infection_case not in causes:
                infection_case = "community infection"
            category = causes.index(infection_case)
            categories.append(category)
        return categories
    
    def type_category(self, types):
        categories = []
        for purpose in types:
            category = move_types.index(purpose)
            categories.append(category)
        return categories
    
    def day_category(self, days):
        categories = []
        for day in days:
            day = datetime.strptime(day, "%Y-%m-%d")
            category = day.weekday()
            categories.append(category)
        return categories
    
    def put_all_categories(self, order, ages, sexes, infection_cases, types, days, lats, lons):
        converter = GridMap()
        length = len(ages)
        
        ages_categories = self.age_category(ages)
        sexes_categories = self.sex_category(sexes)
        infection_case_categories = self.infection_case_category(infection_cases)
        type_categories = self.type_category(types)
        days_categories = self.day_category(days)
        
        # age_categories = 3이라고 했을 때
        # 전체 feature 배열에서 index가 3인 배열에 넣으면 됨
        for i in range(length):
            x, y = converter.toGrid(lats[i], lons[i])
            index = 0
            # age
            age = ages_categories[i]
            self.tensor[order][x][y][age] += 1
            self.features[age][x][y] += 1
            index += self.counts[0]
            # sex
            if sexes_categories[i] != -1:
                sex = index + sexes_categories[i]
                self.tensor[order][x][y][sex] += 1
                self.features[sex][x][y] += 1
                index += self.counts[1]
            # infection_case
            if infection_case_categories[i] != -1:
                infection_case = index + infection_case_categories[i]
                self.tensor[order][x][y][infection_case] += 1
                self.features[infection_case][x][y] += 1
                index += self.counts[2]
            # type
            visit = index + type_categories[i]
            self.tensor[order][x][y][visit] += 1
            self.features[visit][x][y] += 1
            index += self.counts[3]
            # day
            day = index + days_categories[i]
            self.tensor[order][x][y][day] += 1
            self.features[day][x][y] += 1

### 4. 메서드, date 변수

In [7]:
def generate_images(feature_class, merge_route):
    feature_class.clear(len(route_dates))
    images = []

    for i in range(84):
        routes = merge_route.loc[merge_route['date'] == route_dates[i]]

        # features in routes
        ages = routes['age'].tolist()
        sexes = routes['sex'].tolist()
        infection_cases = routes['infection_case'].tolist()
        types = routes['type'].tolist()
        dates = routes['date'].tolist()
        lats = routes['latitude'].tolist()
        lons = routes['longitude'].tolist()

        # features
        feature_class.put_all_categories(i, ages, sexes, infection_cases, types, dates, lats, lons)
#         images.append(feature_class.features)
        
    return feature_class.tensor

In [8]:
route_dates = merge_route['date']
route_dates = route_dates.drop_duplicates(keep='last')
route_dates = route_dates.tolist()

### 5. 피처에 값 대입
##### 방문목적지별 방문 개수
- hospital(970)
- etc(648)
- public transportation(259)
- store(217)
- restuarant(203)
- pharmacy(75)
- church(64)
- cafe(45)
- airport(28)
- pc_cafe(24)
- lodging(24)
- real_estate_agency(16)
- bank(11)
- school(11)
- bar(9)
- beuty_salon(6)
- post_office(5)
- bakery(6)
- gym(3)
- gas_station(3)
- karaoke(2)

##### 피처 병합 계획
- move types의 수를 줄인다.
  - store(217)로 분류될 수 있는 속성들은 store로 대체한다.
    - bakery(6), pharmacy(75) → store
  - restaurant로 분류될 수 있는 속성들은 restuarant(203)로 대체한다.
    - cafe(45), bar(9)
  - 실내 인구 밀집도가 높은 지역은 'high_population'으로 대체한다.
    - karaoke(2), gym(3), pc_cafe(24), church(64)
  - 통계적으로 다른 방문지보다 방문 횟수가 적은 곳은 'etc'로 대체한다.
    - gas_station(3), beuty_salon(6), school(11), real_estate_agency(16), airport(28), bank(11), post_office(5)
  - 남은 것
    - lodging(24), public transportation(259), etc(648), hospital(970), store

### 5. 피처에 값 대입(1)

모든 categorical values를 1개의 피처로 반영

In [9]:
names = ['age', 'sex', 'infection_case', 'type', 'date']
counts = [11, 2, 4, 21, 7]
move_types = ['karaoke', 'gas_station', 'gym', 'bakery', 'pc_cafe',
              'beauty_salon', 'school', 'church', 'bank', 'cafe',
              'bar', 'post_office', 'real_estate_agency', 'lodging',
              'public_transportation', 'restaurant', 'etc', 'store',
              'hospital', 'pharmacy', 'airport']

In [10]:
feature = Feature(names, counts, move_types)
image1 = generate_images(feature, merge_route)

```
images1 = []

for i in range(len(route_dates)):
    routes = merge_route.loc[merge_route['date'] == route_dates[i]]
    
    # features in routes
    ages = routes['age'].tolist()
    sexes = routes['sex'].tolist()
    infection_cases = routes['infection_case'].tolist()
    types = routes['type'].tolist()
    dates = routes['date'].tolist()
    lats = routes['latitude'].tolist()
    lons = routes['longitude'].tolist()
    
    # features
    feature = Feature(names, counts, move_types)
    feature.put_all_categories(ages, sexes, infection_cases, types, dates, lats, lons)
    images1.append(feature)
```

### 5. 피처에 값 대입(2)

방문 목적이 비슷한 항몪끼리 묶는다.
- store(217)로 분류될 수 있는 속성들은 store로 대체한다.
    - bakery(6), pharmacy(75) → store
    - 5월 12일까지 약국에서 일어난 2차 감염 사례가 보고되지 않았기 때문에 약국도 store로 분류

In [11]:
names = ['age', 'sex', 'infection_case', 'type', 'date']
counts = [11, 2, 4, 19, 7]
move_types = ['karaoke', 'gas_station', 'gym', 'pc_cafe',
              'beauty_salon', 'school', 'church', 'bank', 'cafe',
              'bar', 'post_office', 'real_estate_agency', 'lodging',
              'public_transportation', 'restaurant', 'etc', 'store',
              'hospital', 'airport']

In [12]:
second_route = merge_route
second_route['type'].replace({'bakery': 'store', 'pharmacy': 'store'}, inplace=True)

In [13]:
image2 = generate_images(feature, second_route)

### 5. 피처에 값 대입(3)

방문 목적이 비슷한 항몪끼리 묶는다.
- restaurant로 분류될 수 있는 속성들은 restuarant(203)로 대체한다.
  - cafe(45), bar(9)

In [14]:
names = ['age', 'sex', 'infection_case', 'type', 'date']
counts = [11, 2, 4, 17, 7]
move_types = ['karaoke', 'gas_station', 'gym', 'pc_cafe',
              'beauty_salon', 'school', 'church', 'bank',
              'post_office', 'real_estate_agency', 'lodging',
              'public_transportation', 'restaurant', 'etc', 'store',
              'hospital', 'airport']

In [15]:
third_route = second_route
third_route['type'].replace({'cafe': 'restaurant', 'bar': 'restaurant'}, inplace=True)

In [16]:
image3 = generate_images(feature, third_route)

### 5. 피처에 값 대입(4)

실내 환경이 비슷한 항몪끼리 묶는다.
- 실내 인구 밀집도가 높고 폐쇄된 지역은 'high_population'으로 대체한다.
  - karaoke(2), gym(3), pc_cafe(24), church(64)

In [17]:
names = ['age', 'sex', 'infection_case', 'type', 'date']
counts = [11, 2, 4, 14, 7]
move_types = ['gas_station', 'high_population',
              'beauty_salon', 'school', 'bank',
              'post_office', 'real_estate_agency', 'lodging',
              'public_transportation', 'restaurant', 'etc', 'store',
              'hospital', 'airport']

In [18]:
forth_route = third_route
forth_route['type'].replace({'karaoke': 'high_population', 'gym': 'restaurant', 'pc_cafe': 'high_population', 'church': 'high_population'}, inplace=True)

In [19]:
image4 = generate_images(feature, forth_route)

### 5. 피처에 값 대입(5)

- 통계적으로 다른 방문지보다 방문 횟수가 적은 곳은 'etc'로 대체한다.
  - gas_station(3), beuty_salon(6), school(11), real_estate_agency(16), airport(28), bank(11), post_office(5)

In [20]:
names = ['age', 'sex', 'infection_case', 'type', 'date']
counts = [11, 2, 4, 7, 7]
move_types = ['high_population', 'lodging', 'hospital', 'etc',
              'public_transportation', 'restaurant', 'store']

In [21]:
fifth_route = forth_route
fifth_route['type'].replace({'gas_station': 'etc', 'beauty_salon': 'etc', 'school': 'etc', 'real_estate_agency': 'etc', 'airport': 'etc', 'bank': 'etc', 'post_office': 'etc'}, inplace=True)

In [22]:
image5 = generate_images(feature, fifth_route)

### 5. 피처에 값 대입(6)

date에서 평일끼리 묶고, 주말끼리 묶는다.

In [23]:
names = ['age', 'sex', 'infection_case', 'type', 'date']
counts = [11, 2, 4, 7, 2]
move_types = ['high_population', 'lodging', 'hospital', 'etc',
              'public_transportation', 'restaurant', 'store']

In [24]:
sixth_route = fifth_route

class Feature2(Feature):
    def day_category(self, days):
        categories = []
        for day in days:
            day = datetime.strptime(day, "%Y-%m-%d")
            category = day.weekday()
            if category == 0 or category == 6: category = 0
            else: category = 1
            categories.append(category)
        return categories

In [25]:
feature2 = Feature2(names, counts, move_types)
image6 = generate_images(feature2, sixth_route)

### 5. 피처에 값 대입(7)

성별 피처를 삭제한다.

In [26]:
names = ['age', 'sex', 'infection_case', 'type', 'date']
counts = [11, 0, 4, 7, 7]
move_types = ['high_population', 'lodging', 'hospital', 'etc',
              'public_transportation', 'restaurant', 'store']

In [27]:
seventh_route = fifth_route

class Feature3(Feature):
    def sex_category(self, sexes):
        categories = []
        for sex in sexes:
            categories.append(-1)
        return categories

In [28]:
feature3 = Feature3(names, counts, move_types)
image7 = generate_images(feature3, seventh_route)

### 5. 피처에 값 대입(8)

나이대를 통합한다.
- 0s, 10s
- 20s, 30s
- 40s, 50s,
- 60s, 70s,
- 80s, 90s, 100s

In [29]:
names = ['age', 'sex', 'infection_case', 'type', 'date']
counts = [5, 2, 4, 7, 7]
move_types = ['high_population', 'lodging', 'hospital', 'etc',
              'public_transportation', 'restaurant', 'store']

In [30]:
eightth_route = fifth_route

class Feature4(Feature):
    def age_category(self, ages):
        categories = []
        for age in ages:
            age = int(age[:-1])
            if age == 0 or age == 10: category = 0
            elif age == 20 or age == 30: category = 1
            elif age == 40 or age == 50: category = 2
            elif age == 70 or age == 70: category = 3
            else: category = 4
            categories.append(category)
        return categories

In [31]:
feature4 = Feature4(names, counts, move_types)
image8 = generate_images(feature4, eightth_route)

### 5. 피처에 값 대입(9)

감염 원인 피처를 삭제한다.

In [32]:
names = ['age', 'sex', 'infection_case', 'type', 'date']
counts = [5, 2, 0, 7, 7]
move_types = ['high_population', 'lodging', 'hospital', 'etc',
              'public_transportation', 'restaurant', 'store']

In [33]:
nineth_route = fifth_route

class Feature5(Feature):
    def infection_case_category(self, infection_cases):
        categories = []
        for infection_case in infection_cases:
            categories.append(-1)
        return categories

In [34]:
feature5 = Feature5(names, counts, move_types)
image9 = generate_images(feature5, nineth_route)

### 5. 피처에 값 대입(10)

date에서 평일끼리 묶고, 주말끼리 묶는다.
성별 피처를 삭제한다.
나이대를 통합한다.
감염 원인 피처를 삭제한다.

In [35]:
names = ['age', 'sex', 'infection_case', 'type', 'date']
counts = [5, 0, 0, 7, 2]
move_types = ['high_population', 'lodging', 'hospital', 'etc',
              'public_transportation', 'restaurant', 'store']

In [36]:
tenth_route = fifth_route

class Feature6(Feature):
    def day_category(self, days):
        categories = []
        for day in days:
            day = datetime.strptime(day, "%Y-%m-%d")
            category = day.weekday()
            if category == 0 or category == 6: category = 0
            else: category = 1
            categories.append(category)
        return categories
    
    def sex_category(self, sexes):
        categories = []
        for sex in sexes:
            categories.append(-1)
        return categories
    
    def age_category(self, ages):
        categories = []
        for age in ages:
            age = int(age[:-1])
            if age == 0 or age == 10: category = 0
            elif age == 20 or age == 30: category = 1
            elif age == 40 or age == 50: category = 2
            elif age == 70 or age == 70: category = 3
            else: category = 4
            categories.append(category)
        return categories
    
    def infection_case_category(self, infection_cases):
        categories = []
        for infection_case in infection_cases:
            categories.append(-1)
        return categories

In [37]:
feature6 = Feature6(names, counts, move_types)
image10 = generate_images(feature6, tenth_route)

In [38]:
image10.shape

(92, 256, 256, 14)

### 6. ConvLSTM

In [39]:
from keras.models import Sequential
from keras.layers.convolutional import Conv3D
from keras.layers.convolutional_recurrent import ConvLSTM2D
from keras.layers.normalization import BatchNormalization
import keras.backend.tensorflow_backend as K
import numpy as np
import pylab as plt

Using TensorFlow backend.

Bad key "text.kerning_factor" on line 4 in
C:\Users\Yejin Kim\anaconda3\envs\AutoCOVID19\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


In [46]:
# We create a layer which take as input movies of shape
# (n_frames, width, height, channels) and returns a movie
# of identical shape.
def get_model(number, channel):
    with K.tf_ops.device('/device:GPU:0'):
        seq = Sequential()
        seq.add(ConvLSTM2D(filters=128, kernel_size=(3, 3),
                           input_shape=(14, 256, 256, channel),
                           padding='same', return_sequences=True))
        seq.add(BatchNormalization())

        seq.add(ConvLSTM2D(filters=128, kernel_size=(3, 3),
                           padding='same', return_sequences=True))
        seq.add(BatchNormalization())

        seq.add(ConvLSTM2D(filters=128, kernel_size=(3, 3),
                           padding='same', return_sequences=True))
        seq.add(BatchNormalization())

        seq.add(ConvLSTM2D(filters=128, kernel_size=(3, 3),
                           padding='same', return_sequences=True))
        seq.add(BatchNormalization())

        seq.add(Conv3D(filters=channel, kernel_size=(3, 3, 3),
                       activation='sigmoid',
                       padding='same', data_format='channels_last'))

        seq.compile(loss='binary_crossentropy', optimizer='adadelta')
    return seq

#### train/test

In [41]:
image1.shape

(92, 256, 256, 45)

In [42]:
n = image1.shape[0]
rs = 256
n_step = 14

n_test = n_step * 2
n_train = n - n_test

train = image1[:n_train,:,:]
test = image1[n_train:,:,:]

print(train.shape)
print(test.shape)

(64, 256, 256, 45)
(28, 256, 256, 45)


In [43]:
X_train = []
y_train = []
for i in range(n_step,n_train-n_step):
    X_train.append(train[i-n_step:i, :,:])
    y_train.append(train[i:i+n_step, :,:])
X_train, y_train = np.array(X_train), np.array(y_train)

print(X_train.shape)
print(y_train.shape)

(36, 14, 256, 256, 45)
(36, 14, 256, 256, 45)


In [47]:
seq = get_model(0, image1.shape[3])

In [48]:
seq.fit(X_train, y_train, epochs = 50, batch_size = 32)

Epoch 1/50


MemoryError: Unable to allocate 9.84 GiB for an array with shape (32, 14, 256, 256, 45) and data type float64

In [ ]:
train=train.reshape((train.shape[0],n_step,rs,rs,45))
print(train.shape)
test=test.reshape((test.shape[0],rs,rs,1))
print(test.shape)

In [ ]:
---
# (n_frames, width, height, channels) and returns a movie

(17484, 12, 50, 50)
(17484, 1, 50, 50)

(17484, 12, 50, 50, 1)
(17484, 50, 50, 1)

In [ ]:
X_train=X_train.reshape((X_train.shape[0],14,256,256,45))
print(X_train.shape)
y_train=y_train3.reshape((y_train.shape[0],256,256,45))
print(y_train.shape)

In [ ]:
seq = get_model(0, image1.shape[2])

In [ ]:
seq.summary()

In [ ]:
history = seq.fit(noisy_movies[:1000], shifted_movies[:1000], batch_size=10,
        epochs=50, validation_split=0.05)

In [ ]:
model_dir = './model'
if not os.path.exists(model_dir):
    os.mkdir(model_dir)

model_path = model_dir + '/covid%d.h5' % number
seq.save(model_path)

In [ ]:
print("정확도 : %.4f" % (model.evaluate()[1]))
print("손실 : %.4f" % (model.evaluate()[0]))